# Helheim Glacier

In this demo we'll look at Helheim Glacier in southeast Greenland.
Helheim is one of the bigger glaciers draining the Greenland Ice Sheet by mass flux along with Jakobshavn, Kangerdlugssuaq, Petermann, and the Northeast Greenland Ice Stream.
Much of the procedure for working with real data should be familiar from the demo on [inverse problems](https://icepack.github.io/icepack.demo.04-ice-shelf-inverse.html), where we inferred the fluidity of the Larsen C Ice Shelf in the Antarctic Peninsula.
Here we'll use many of the same techniques, only instead of inferring the fluidity, we'll look at the friction coefficient for glacier sliding over the underlying bedrock.
In the following, we'll assume that ice flow is by Weertman sliding:

$$\tau_b = -C|u|^{\frac{1}{m} - 1}u,$$

where $\tau_b$ is the basal shear stress, $u$ is the sliding velocity, $m \approx 3$ is the friction exponent, and $C$ is the friction coefficient.
First, we'll estimate the friction coefficient using a 2D, depth-averaged model.
Then we'll use this solution as an initial guess using the 3D hybrid model that we showed in [this demo](https://icepack.github.io/icepack.demo.05-hybrid-ice-stream.html) for a synthetic ice stream.

### Input data

Loading in the mesh and input data should be mostly familiar from the previous demos on Larsen C.
We'll be using different data sets since we're looking at Greenland rather than Antarctica this time around.

In [ ]:
import geojson
import icepack
outline_filename = icepack.datasets.fetch_outline('helheim')
with open(outline_filename, 'r') as outline_file:
    outline = geojson.load(outline_file)

Next we'll calculate a bounding box around the outline.

In [ ]:
import numpy as np
xmin, ymin, xmax, ymax = np.inf, np.inf, -np.inf, -np.inf
δ = 5e3
for feature in outline['features']:
    for line_string in feature['geometry']['coordinates']:
        xs = np.array(line_string)
        x, y = xs[:, 0], xs[:, 1]
        xmin, ymin = min(xmin, x.min() - δ), min(ymin, y.min() - δ)
        xmax, ymax = max(xmax, x.max() + δ), max(ymax, y.max() + δ)

Like we did for Larsen, we'll visualize everything on top of a satellite image mosaic of Greenland.
The official reason to do this is that it helps us to make sure that all the coordinate systems make sense, and the actual reason is that we just want to make prettier plots.
Once again we'll need to do a windowed read because we only care about the area around Helheim.

In [ ]:
import rasterio
import icepack.plot
image_filename = icepack.datasets.fetch_mosaic_of_greenland()
image_file = rasterio.open(image_filename, 'r')

height, width = image_file.height, image_file.width
transform = image_file.transform
window = rasterio.windows.from_bounds(
    left=xmin, bottom=ymin, right=xmax, top=ymax,
    width=width, height=height, transform=transform
)

image = image_file.read(indexes=1, window=window, masked=True)

def subplots(*args, **kwargs):
    fig, axes = icepack.plot.subplots()
    xmin, ymin, xmax, ymax = rasterio.windows.bounds(window, transform)
    axes.imshow(
        image,
        cmap='Greys_r',
        vmin=8e3,
        vmax=16.38e3,
        extent=(xmin, xmax, ymin, ymax)
    )
    
    return fig, axes

Now we'll turn the shape file of the outline into the input format for gmsh and generate a mesh.

In [ ]:
geometry = icepack.meshing.collection_to_geo(outline, lcar=5e3)
with open('helheim.geo', 'w') as geo_file:
    geo_file.write(geometry.get_code())

In [ ]:
!gmsh -2 -format msh2 -v 2 -o helheim.msh helheim.geo

The initial mesh that we've generated is pretty coarse, so to get finer resolution we'll create a hierarchy of refined meshes.
In our case the mesh hierarchy will only go one level deep, so each triangle has been cut into four similar triangles.
Where possible, it's best to start with the coarsest mesh and refine up to the level you need; you can always make a small problem bigger, but it's hard to make a big problem smaller.

In [ ]:
import firedrake
mesh = firedrake.Mesh('helheim.msh')

In [ ]:
fig, axes = subplots()
kwargs = {
    'interior_kw': {'linewidth': 0.1},
    'boundary_kw': {'linewidth': 2}
}
icepack.plot.triplot(mesh, axes=axes, **kwargs)
axes.legend();

Now we can start loading in all the observational data and interpolating it to the function spaces we'll use.

In [ ]:
thickness_filename = icepack.datasets.fetch_bedmachine_greenland()
thickness = rasterio.open(f'netcdf:{thickness_filename}:thickness', 'r')
surface = rasterio.open(f'netcdf:{thickness_filename}:surface', 'r')

In [ ]:
degree = 2
Q = firedrake.FunctionSpace(mesh, 'CG', degree)
h_obs = icepack.interpolate(thickness, Q)
s_obs = icepack.interpolate(surface, Q)

In [ ]:
from firedrake import inner, grad, dx
def smooth(q_obs, α):
    q = q_obs.copy(deepcopy=True)
    J = 0.5 * ((q - q_obs)**2 + α**2 * inner(grad(q), grad(q))) * dx
    F = firedrake.derivative(J, q)
    firedrake.solve(F == 0, q)
    return q

α = firedrake.Constant(2e3)
h = smooth(h_obs, α)
s = smooth(s_obs, α)

In [ ]:
fig, axes = subplots()
colors = icepack.plot.tripcolor(h, axes=axes)
fig.colorbar(colors);

In [ ]:
fig, axes = subplots()
colors = icepack.plot.tripcolor(s, axes=axes)
fig.colorbar(colors);

In [ ]:
velocity_filenames = icepack.datasets.fetch_measures_greenland()
velocity_dict = {
    key: [f for f in velocity_filenames if key in f][0]
    for key in ['vx', 'vy', 'ex', 'ey']
}
vx = rasterio.open(velocity_dict['vx'], 'r')
vy = rasterio.open(velocity_dict['vy'], 'r')
ex = rasterio.open(velocity_dict['ex'], 'r')
ey = rasterio.open(velocity_dict['ey'], 'r')

In [ ]:
V = firedrake.VectorFunctionSpace(mesh, 'CG', degree)
u_obs = icepack.interpolate((vx, vy), V)
σx = icepack.interpolate(ex, Q)
σy = icepack.interpolate(ey, Q)

In [ ]:
from firedrake import min_value
σx = firedrake.interpolate(min_value(200, abs(σx)), Q)
σy = firedrake.interpolate(min_value(200, abs(σy)), Q)

In [ ]:
σx.dat.data_ro[:].min()
σy.dat.data_ro[:].min()

In [ ]:
fig, axes = subplots()
σ = firedrake.interpolate(firedrake.sqrt(σx**2 + σy**2), Q)
colors = icepack.plot.tripcolor(σ, vmax=50, axes=axes)
fig.colorbar(colors);

Now that we've loaded in the ice thickness and surface elevation, we'll calculate the gravitational driving stress

$$\tau_d = -\rho_Igh\nabla s.$$

The gravitational driving stress is in balance with internal viscous stresses and with basal stress.
By knowing roughly what the magnitudes of the driving stress and the velocity are, we can get a better idea of what a reasonable starting value of the friction coefficient should be.

In [ ]:
from icepack.constants import ice_density as ρ_I, gravity as g
τ = firedrake.project(-ρ_I * g * h * grad(s), V)

In [ ]:
fig, axes = subplots()
colors = icepack.plot.tripcolor(τ, vmax=0.5, axes=axes)
fig.colorbar(colors, label='megapascals');

To initialize the inverse problem, we'll assume that the basal shear stress takes up half of the driving stress.
This is a totally ad hoc assumption and it's not obvious a priori that this is going to give us reasonable values of the ice velocity.
Nonetheless, it happens to work well enough as a starting point.
We'll also need to re-parameterize the basal shear stress in terms of some auxiliary variable $q$ in order to guarantee that the friction coefficient is strictly positive.
In the following, we'll use

$$\tau_b = -\frac{\tau_0}{u_0^{1/m}}e^{-q}|u|^{\frac{1}{m} - 1}u$$

where $\tau_0$ and $u_0$ are the average magnitudes of the driving stress and speed respectively.

In [ ]:
from firedrake import exp, ln, sqrt, assemble
from icepack.constants import weertman_sliding_law as m
import icepack.models.friction

u = u_obs.copy(deepcopy=True)
speed = sqrt(inner(u, u))
stress = sqrt(inner(τ, τ))

area = assemble(firedrake.Constant(1) * dx(mesh))
speed_avg = assemble(speed * dx) / area
stress_avg = assemble(stress * dx) / area

print(f'Average speed:  {speed_avg:6.1f} meters / year')
print(f'Average stress: {1e3 * stress_avg:6.1f} kilopascals')

fraction = 0.5
C = fraction * stress / speed**(1/m)
q = firedrake.interpolate(-ln(speed_avg**(1/m) * C / stress_avg), Q)

def bed_friction(**kwargs):
    u, q = map(kwargs.get, ('velocity', 'log_friction'))
    
    C = stress_avg / speed_avg**(1/m) * exp(-q)
    return icepack.models.friction.bed_friction(velocity=u, friction=C)

For now, we'll assume that the ice is at a constant -13C.
We'll revisit this assumption using a heat flow model later.

In [ ]:
T = firedrake.Constant(260.)
A = icepack.rate_factor(T)

In [ ]:
flow_model = icepack.models.IceStream(friction=bed_friction)
opts = {'dirichlet_ids': [1, 2, 4, 5, 6]}
flow_solver = icepack.solvers.FlowSolver(flow_model, **opts)
u = flow_solver.diagnostic_solve(
    velocity=u_obs, 
    thickness=h, 
    surface=s, 
    fluidity=A, 
    log_friction=q
)

In [ ]:
fig, axes = subplots()
contours = icepack.plot.tricontourf(u, axes=axes)
fig.colorbar(contours, label='meters / year');

### Inferring the friction

Next we'll apply a similar procedure as the first demo on inverse problems to back out the friction coefficient.
The first ingredient is to define the objective and regularization functionals.
Rather than use a least-squares type functional to measure the model-data misfit, we'll include a fudge factor that makes the fitting procedure taper off at large misfit values.
When the observational data are not actually normally distributed, least-squares fitting can focus too much on outlier data points and, in so doing, spoil the overall result.

In [ ]:
ϵ = firedrake.Constant(1.)
def objective(u):
    δu = u - u_obs
    χ2 = (δu[0] / σx)**2 + (δu[1] / σy)**2
    return (sqrt(χ2 + ϵ**2) - ϵ) * dx

The regularization functional is exactly as before.
We're more interested in using this demo to explore 3D flow and heat transport, so it's not essential to resolve the basal shear stress down to the most minute details.
As a consequence we can get away with using a fairly large regularization parameter.

In [ ]:
Φ = firedrake.Constant(1)
L = firedrake.Constant(5e3)
def regularization(q):
    return 0.5 * (L / Φ)**2 * inner(grad(q), grad(q)) * dx

The formulation of the inverse problem is very similar to what we saw in the previous tutorial for the Larsen ice shelf, but we have to pass in a few extra fields to the diagnostic solver.

In [ ]:
problem = icepack.inverse.InverseProblem(
    model=flow_model,
    objective=objective,
    regularization=regularization,
    state_name='velocity',
    state=u,
    parameter_name='log_friction',
    parameter=q,
    solver_kwargs=opts,
    diagnostic_solve_kwargs={
        'thickness': h,
        'surface': s,
        'fluidity': A
    }
)

In [ ]:
parameters = {'form_compiler_parameters': {'quadrature_degree': 2 * degree + 1}}
def callback(solver):
    dJ = solver.gradient
    q = solver.search_direction
    dJ_dq = firedrake.action(dJ, q)
    Δ = firedrake.assemble(dJ_dq, **parameters)

    E = firedrake.assemble(solver.objective)
    R = firedrake.assemble(solver.regularization)
    print(f'{E / area:g}, {R / area:g}, {Δ / area:g}')

This test case is more challenging for the inverse solver than the example we used to demonstrate how the inverse solver works in the first place.
We'll need to increase the maximum number of iterations of the conjugate gradient algorithm in order to guarantee that the linear solver for the search direction will actually converge.
To do this, we'll pass a value for the keyword argument `'search_max_iterations'` to the inverse solver.
Steps like this one are often necessary for especially challenging problems.

In [ ]:
solver = icepack.inverse.GaussNewtonSolver(
    problem,
    callback,
    search_max_iterations=200
)

Once again, as a sanity check we'll plot the initial search direction.

In [ ]:
fig, axes = subplots()
ϕ = solver.search_direction
colors = icepack.plot.tripcolor(ϕ, cmap='RdBu', axes=axes)
fig.colorbar(colors);

Descending along this direction will reduce the basal shear stress along the main trunk of the glacier, which makes sense given that outlet glaciers tend to be found where weak sediments facilitate fast sliding.
Now let's go get a coffee and wait for the solver to converge.

In [ ]:
iterations = solver.solve(
    rtol=5e-3,
    atol=0.0,
    etol=1e-4,
    max_iterations=30
)

In [ ]:
fig, axes = subplots()
q = solver.parameter
colors = icepack.plot.tripcolor(
    q, vmin=-5, vmax=+5, cmap='bwr', axes=axes
)
fig.colorbar(colors);

In [ ]:
u = solver.state

C = stress_avg / speed_avg**(1/m) * exp(-q)
U = sqrt(inner(u, u))
τ_b = firedrake.interpolate(C * U**(1/m), Q)

In [ ]:
fig, axes = subplots()
colors = icepack.plot.tripcolor(
    τ_b, vmin=0, vmax=0.5, axes=axes
)
fig.colorbar(colors, label='megapascals');

### Lifting to 3D

We looked at modeling the flow of ice streams using the hybrid model in a previous tutorial, but there we were using purely synthetic data and we started out working with 3D fields to begin with.
Here we've used a 2D flow model to more cheaply calculate the basal shear stress, and we'd now like to transfer these results to a 3D domain.
The convenience function `lift3d` will do this for us.

In [ ]:
help(icepack.lift3d)

First, we'll extrude the mesh and create a function space to represent scalar fields that are constant throughout the whole column.
Here we'll use the `R` space in the vertical, which represents spatially constant functions (see also [this page](https://firedrakeproject.org/r-space.html#using-r-space-with-extruded-meshes) from the Firedrake documentation).

In [ ]:
mesh3d = firedrake.ExtrudedMesh(mesh, 1)
Qc = firedrake.FunctionSpace(
    mesh3d, family='CG', degree=2, vfamily='R', vdegree=0
)
q = icepack.lift3d(solver.parameter, Qc)
h = icepack.lift3d(h, Qc)
s = icepack.lift3d(s, Qc)
σx = icepack.lift3d(σx, Qc)
σy = icepack.lift3d(σy, Qc)

Next, we need to lift the 2D depth-averaged velocity into 3D.
This takes place in two stages, the first of which is to extrude the velocity to be constant in the vertical direction.
The initial function space `V0` will use degree-0 elements in the vertical.

In [ ]:
V0 = firedrake.VectorFunctionSpace(
    mesh3d, dim=2, family='CG', degree=2, vfamily='R', vdegree=0
)
u0 = icepack.lift3d(solver.state, V0)
u_obs = icepack.lift3d(u_obs, V0)

Next, we'll add vertical variation.
The function space `V` that we'll use for the velocities has polynomial degree 2 in the vertical.
This is sufficiently many degrees of freedom to resolve both the zero-stress boundary condition at the ice surface and the sliding boundary condition at the ice base.

In [ ]:
V = firedrake.VectorFunctionSpace(
    mesh3d, dim=2, family='CG', degree=2, vfamily='GLL', vdegree=2
)

x, y, ζ = firedrake.SpatialCoordinate(mesh3d)
u_initial = firedrake.interpolate((1 - 0.5 * (1 - ζ)**2) * u0, V)

Finally, we'll create model and solver objects and calculate a 3D velocity field.

In [ ]:
def bed_friction(**kwargs):
    u, q = map(kwargs.get, ('velocity', 'log_friction'))

    C = stress_avg / speed_avg**(1/m) * exp(-q)
    return icepack.models.hybrid.bed_friction(velocity=u, friction=C)

flow_model = icepack.models.HybridModel(friction=bed_friction)

In [ ]:
opts = {
    'dirichlet_ids': [1, 2, 4, 5, 6],
    'diagnostic_solver_type': 'petsc',
    'diagnostic_solver_parameters': {
        'snes_type': 'newtontr',
        'pc_type': 'lu',
        'pc_factor_mat_solver_type': 'mumps'
    }
}
flow_solver = icepack.solvers.FlowSolver(flow_model, **opts)

In [ ]:
fields = dict(
    velocity=u_initial,
    thickness=h,
    surface=s,
    fluidity=A,
    log_friction=q
)

In [ ]:
qdegree = flow_model.quadrature_degree(**fields)

In [ ]:
u = flow_solver.diagnostic_solve(
    velocity=u_initial,
    thickness=h,
    surface=s,
    fluidity=A,
    log_friction=q
)

Since we're no longer working purely in 2D, we'll want to get some idea of the vertical variation of the computed velocity.
If we just pass $u$ itself to the plotting functions like `tricontourf`, we'll get a depth average of $u$, which we've already seen above.
Instead, let's try and learn more about the vertical structure of the velocity field.
If we pass a *weight* to the `depth_average` function, we can calculate all sorts of different things besides the actual depth average.
For example, if $P_n$ are the shifted Legendre polynomials, we can calculate the basal velocity by multiplying the 3D velocity $u$ by the function

$$q(\zeta) = \sum_n p_n(0)\cdot p_n(\zeta)$$

and integrating in $\zeta$ from 0 to 1.
The first few normalized, shifted Legendre polynomials are

$$\begin{align}
P_0 & = 1 \\
P_1 & = \sqrt{3}\cdot(2\zeta - 1) \\
P_2 & = \sqrt{5}\cdot(6\zeta^2 - 6\zeta + 1)
\end{align}$$

from which we can easily read off the values around 0.

In [ ]:
P_0 = 1
P_1 = np.sqrt(3) * (2 * ζ - 1)
P_2 = np.sqrt(5) * (6 * ζ**2 - 6 * ζ + 1)

weight = P_0 - np.sqrt(3) * P_1 + np.sqrt(5) * P_2
u_b = icepack.depth_average(u, weight=weight)

We can do the same thing to read off the values at the ice surface ($\zeta = 1$).

In [ ]:
weight = P_0 + np.sqrt(3) * P_1 + np.sqrt(5) * P_2
u_s = icepack.depth_average(u, weight=weight)

Now that we've calculated both the basal and surface velocities, we can look at the ratio between them.
The sliding ratio is close to 1 in the fastest-flowing parts of the glacier where the bed provides almost no resistance.
The sliding ratio is more like to 1/2 further upstream and gets close t0 0 around highly resistive patches, for example around the nunatak just to the west of the main glacier trunk.

In [ ]:
U_b = sqrt(inner(u_b, u_b))
U_s = sqrt(inner(u_s, u_s))
ratio = firedrake.project(U_b / U_s, Q)

In [ ]:
fig, axes = subplots()
colors = icepack.plot.tripcolor(
    ratio, vmin=0.0, vmax=1.0, cmap='Blues', axes=axes
)
fig.colorbar(colors);

### Heat transport

Now let's see if we can use the velocity and basal shear stress fields to compute the ice temperature.

In [ ]:
Q = firedrake.FunctionSpace(
    mesh3d, family='CG', degree=2, vfamily='GLL', vdegree=2
)
heat_transport = icepack.models.HeatTransport3D()

In [ ]:
T = firedrake.interpolate(firedrake.Constant(260.0), Q)
f = firedrake.interpolate(firedrake.Constant(0.0), Q)
E = firedrake.interpolate(heat_transport.energy_density(T, f), Q)
A = icepack.rate_factor(T)

In [ ]:
from icepack.models.hybrid import horizontal_strain, vertical_strain, stresses
ε_x = horizontal_strain(u, s, h)
ε_z = vertical_strain(u, h)
M_x, M_z = stresses(ε_x, ε_z, A)
dQ = inner(M_x, ε_x) + inner(M_z, ε_z)

In [ ]:
P = bed_friction(velocity=u, log_friction=q)
τ_b = firedrake.derivative(P, u)
dQ_b = inner(u, tau_b)

In [ ]:
heat_solver = icepack.models.HeatTransportSolver(heat_transport)

Now by some miracle we'll compute the vertical velocity...

In [ ]:
Q_w = firedrake.FunctionSpace(
    mesh3d, family='DG', degree=1, vfamily='GLL', degree=3
)
# Miracle occurs...

In [ ]:
final_time = 100.0
timestep = 0.5
num_steps = int(final_time / timestep)
dt = final_time / num_steps

for step in range(num_steps):
    E = heat_solver.solve(
        dt,
        energy=E,
        velocity=u,
        vertical_velocity=w,
        thickness=h,
        surface=s,
        heat=dQ,
        heat_bed=dQ_b,
        energy_inflow=E_initial
        energy_surface=E_surface
    )